In [38]:
import requests
from abc import ABC, abstractmethod
from typing import List, Optional

In [39]:
import importlib

import model
importlib.reload(model)

<module 'model' from 'c:\\Users\\Site Oracle\\reduceComplexity\\model.py'>

In [40]:
aapl = model.Company("Apple", "AAPL", model.SECFilingRepository())

In [41]:
aapl.cik

'0000320193'

In [42]:
print(aapl.filings)

getting filings
filings retrieved
returning filings
[<model.Filing object at 0x0000029A93986890>, <model.Filing object at 0x0000029A9309B310>, <model.Filing object at 0x0000029A93986350>, <model.Filing object at 0x0000029A93986710>, <model.Filing object at 0x0000029A93986850>, <model.Filing object at 0x0000029A93985850>, <model.Filing object at 0x0000029A939859D0>, <model.Filing object at 0x0000029A93985950>, <model.Filing object at 0x0000029A93985A50>, <model.Filing object at 0x0000029A93985810>, <model.Filing object at 0x0000029A93985A90>, <model.Filing object at 0x0000029A93985910>, <model.Filing object at 0x0000029A939879D0>, <model.Filing object at 0x0000029A939858D0>, <model.Filing object at 0x0000029A93985990>, <model.Filing object at 0x0000029A93985710>, <model.Filing object at 0x0000029A93985450>, <model.Filing object at 0x0000029A93984090>, <model.Filing object at 0x0000029A93985050>, <model.Filing object at 0x0000029A93985090>, <model.Filing object at 0x0000029A93985010>, <m

In [43]:
for filing in aapl.filings:
    if filing.form == "10-K":
        test_form = filing
        break

returning filings


In [44]:
df = test_form.income_statement.get_annual_data()

In [45]:
df

,metric,value,start_date,end_date,unit,decimals,segment_value,segment_dimension,period_length
6,RevenueFromContractWithCustomerExcludingAssess...,3.910350e+11,2023-10-01,2024-09-28,usd,-6,None,None,363
7,RevenueFromContractWithCustomerExcludingAssess...,3.832850e+11,2022-09-25,2023-09-30,usd,-6,None,None,370
8,RevenueFromContractWithCustomerExcludingAssess...,3.943280e+11,2021-09-26,2022-09-24,usd,-6,None,None,363
51,CostOfGoodsAndServicesSold,2.103520e+11,2023-10-01,2024-09-28,usd,-6,None,None,363
52,CostOfGoodsAndServicesSold,2.141370e+11,2022-09-25,2023-09-30,usd,-6,None,None,370
53,CostOfGoodsAndServicesSold,2.235460e+11,2021-09-26,2022-09-24,usd,-6,None,None,363
54,GrossProfit,1.806830e+11,2023-10-01,2024-09-28,usd,-6,None,None,363
55,GrossProfit,1.691480e+11,2022-09-25,2023-09-30,usd,-6,None,None,370
56,GrossProfit,1.707820e+11,2021-09-26,2022-09-24,usd,-6,None,None,363
57,ResearchAndDevelopmentExpense,3.137000e+10,2023-10-01,2024-09-28,usd,-6,None,None,363


In [46]:
df[df['segment_value'] == None]

,metric,value,start_date,end_date,unit,decimals,segment_value,segment_dimension,period_length
